In [6]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
from astropy import wcs

The Simple Imaging Polynomial (SIP) convention by Shupe et al. (2005) (http://fits.gsfc.nasa.gov/registry/sip/SIP_distortion_v1_0.pdf) implements distortion corrections as polynomials in FITS headers.

The polynomials are defined as: $${r_x \choose r_y} = \textrm{CD} {x+p_x \choose y+p_y}$$
Here, $\textrm{CD}$ is the $2\times2$ transformation matrix, made up of the $\textrm{CD1_1}$, $\textrm{CD1_2}$, $\textrm{CD2_1}$ and $\textrm{CD2_2}$ parameters. $r_x$ and $r_y$ are the intermediate world coordinates, computed from deprojecting right ascension $\alpha$ and declination $\delta$ around some reference point $\alpha_0$ and $\delta_0$, set by the $\textrm{CRVAL1}$ and $\textrm{CRVAL2}$ keywords. $x$ and $y$ are pixel coordinates, with origin $\textrm{CRPIX1}$ and $\textrm{CRPIX2}$, while $p_x(x, y)$ and $p_y(x, y)$ are the distortion polynomials.

The distortion polynomials are defined as:
$p_x(x, y) = \sum_{i, j} A_{i,j}x^i y^j$ and $p_y(x, y) = \sum_{i, j} B_{i,j}x^i y^j$, with $i+j\le \textrm{A_ORDER}$ and $i+j\le \textrm{B_ORDER}$.

A 3rd order polynomial would have the form: $p_x(x, y) = A_{00}+A_{10}x+A_{20}x^2+A_{30}x^3+A_{01}y+A_{11}xy+A_{21}x^2y+A_{02}y^2+A_{12}xy^2+A_{03}y^3$, as the values for $A_{31}$, $A_{32}$, $A_{33}$, $A_{22}$, $A_{23}$ and $A_{13}$ are all zero. Hence, the polynomial coefficients can be stored as an $(m+1)\times (m+1)$ matrix, where $m = \textrm{A_ORDER}$ and $m=\textrm{B_ORDER}$. Also note that $A_{00}$, $A_{01}$ and $A_{10}$ are zero, as these parameters are dealt with by the $\textrm{CD}$ matrix.

Given that the $\textrm{CD}$ matrix is involved, the full relation becomes:
$r_x = \textrm{CD1_1} (x+p_x(x, y)) + \textrm{CD1_2} (y+p_y(x, y))$, such that:
$$r_x = \textrm{CD1_1} x + \textrm{CD1_1}A_{02}x^2+\textrm{CD1_1}A_{11}xy+\textrm{CD1_1}A_{20}x^2$$

In [7]:
t = ascii.read("distort.dat")

In [8]:
ra0, dec0 = np.mean(t["ra"]), np.mean(t["dec"])
x0, y0 = 1548, 1040
dx, dy = t["x"]-x0, t["y"]-y0

w = wcs.WCS(naxis=2)
w.wcs.ctype = ["RA---ZEA", "DEC--ZEA"]
w.wcs.cd = [[-1.0, 0.0], [0.0, 1.0]]
w.wcs.crval = [ra0, dec0]
w.wcs.crpix = [0.0, 0.0]

p = w.wcs_world2pix(t["ra"], t["dec"], 1)
rx, ry = p[0], p[1]

In [24]:
n = len(t)
m = 6
p = np.ones(m*n).reshape(m, n)
p[1] *= dx
p[2] *= dy
p[3] *= dx*dx
p[4] *= dx*dy
p[5] *= dy*dy

In [25]:
q, r = np.linalg.qr(p)

In [29]:
r.shape

(6, 1198)